In [129]:
import numpy as np

class RBFN:
    def __init__(self, k, lr=0.1, epochs=1000):
        self.k = k # number of centroids
        self.lr = lr # learning rate for LMS
        self.epochs = epochs # number of epochs for LMS
        self.centroids = None
        self.variances = None
        self.weights = None

    def k_means_clustering(self, X):
        # Initialize k centroids randomly
        centroids = X[np.random.choice(X.shape[0], self.k, replace=False)]
        prev_centroids = None
        
        while not np.array_equal(centroids, prev_centroids):
            # Assign each data point to the nearest centroid
            clusters = []
            for j in range(self.k):
                clusters.append([])
            for x in X:
                distances = [np.linalg.norm(x - c) for c in centroids]
                nearest_centroid_idx = np.argmin(distances)
                clusters[nearest_centroid_idx].append(x)

            # Update the centroids to be the mean of each cluster
            prev_centroids = centroids.copy()
            for j in range(self.k):
                if clusters[j]:
                    centroids[j] = np.mean(clusters[j], axis=0)
        
        self.centroids = centroids
        self.variances = np.zeros((self.k,1))
        for j in range(self.k):
            if clusters[j]:
                self.variances[j] = np.mean([np.linalg.norm(x - self.centroids[j]) for x in clusters[j]])
    
    def lms(self, X, y):
        # Initialize the weights randomly
        self.weights = np.random.randn(self.k)
        
        for epoch in range(self.epochs):
            for i, x in enumerate(X):
                # Calculate the RBFN output for the input x
                phi = np.exp(-np.sum((x - self.centroids)**2 / (2 * self.variances**2), axis=1))
                output = np.dot(phi, self.weights)
                
                # Calculate the error and update the weights
                error = y[i] - output
                self.weights += self.lr * error * phi
                
    def fit(self, X, y):
        self.k_means_clustering(X)
        self.lms(X, y)
        
    def predict(self, X):
        # Calculate the RBFN output for each input in X
        phi = np.exp(-np.sum((X[:, np.newaxis] - self.centroids)**2 / (2 * self.variances**2), axis=2))
        output = np.dot(phi, self.weights)
        return np.round(output).astype(int)#Round the output to the neareat integer to give us the lables 

In [132]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create an RBFN with 10 centroids
rbfn = RBFN(k=10)

# Train the RBFN on the training data
rbfn.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rbfn.predict(X_test)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9111111111111111
